# Starbucks dataset

### Introduction

This data set contains simulated data that mimics customer behavior on the Starbucks rewards mobile app. Once every few days, Starbucks sends out an offer to users of the mobile app. An offer can be merely an advertisement for a drink or an actual offer such as a discount or BOGO (buy one get one free). Some users might not receive any offer during certain weeks. 

This data set is a simplified version of the real Starbucks app because the underlying simulator only has one product whereas Starbucks actually sells dozens of products.

Every offer has a validity period before the offer expires. As an example, a BOGO offer might be valid for only 5 days. You'll see in the data set that informational offers have a validity period even though these ads are merely providing information about a product; for example, if an informational offer has 7 days of validity, you can assume the customer is feeling the influence of the offer for 7 days after receiving the advertisement.

### Example

To give an example, a user could receive a discount offer buy 10 dollars get 2 off on Monday. The offer is valid for 10 days from receipt. If the customer accumulates at least 10 dollars in purchases during the validity period, the customer completes the offer.

However, there are a few things to watch out for in this data set. Customers do not opt into the offers that they receive; in other words, a user can receive an offer, never actually view the offer, and still complete the offer. For example, a user might receive the "buy 10 dollars get 2 dollars off offer", but the user never opens the offer during the 10 day validity period. The customer spends 15 dollars during those ten days. There will be an offer completion record in the data set; however, the customer was not influenced by the offer because the customer never viewed the offer.

# Objective: Predicts whether or not someone will respond to an offer

# Data Sets

The data is contained in three files:

* portfolio.json - containing offer ids and meta data about each offer (duration, type, etc.)
* profile.json - demographic data for each customer
* transcript.json - records for transactions, offers received, offers viewed, and offers completed

Here is the schema and explanation of each variable in the files:

**portfolio.json**
* id (string) - offer id
* offer_type (string) - type of offer ie BOGO, discount, informational
* difficulty (int) - minimum required spend to complete an offer
* reward (int) - reward given for completing an offer
* duration (int) - time for offer to be open, in days
* channels (list of strings)

**profile.json**
* age (int) - age of the customer 
* became_member_on (int) - date when customer created an app account
* gender (str) - gender of the customer (note some entries contain 'O' for other rather than M or F)
* id (str) - customer id
* income (float) - customer's income

**transcript.json**
* event (str) - record description (ie transaction, offer received, offer viewed, etc.)
* person (str) - customer id
* time (int) - time in hours since start of test. The data begins at time t=0
* value - (dict of strings) - either an offer id or transaction amount depending on the record

**Note:** You may need to go to the terminal and run the command `conda update pandas` before reading in the files. This is because of the version of pandas of some workspace cannot read in the transcript.json file correctly, but the newest version of pandas can.

# Import The libraries

In [1]:
import pandas as pd
import numpy as np
import math
import json

import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
%matplotlib inline

# Load The Dataset

In [2]:
# read in the json files
portfolio = pd.read_json('data/portfolio.json', orient='records', lines=True)
profile = pd.read_json('data/profile.json', orient='records', lines=True)
transcript = pd.read_json('data/transcript.json', orient='records', lines=True)

# Functions

In [5]:
# Cleaning function
def id_mapper(df, column_name, remove_col=False):
    """ 
    Map id with number
    
    Args:
    df: dataframe. The pandas dataframe.
    column_name: string. The column name.
    remove_col: bool. Remove the orginal column or not
    
    Returns:
    offerid_encoded: The removed outliner dataframe by using z score.
    coded_dict: The coded dictionary
    
    """
    coded_dict = dict()
    cter = 1
    id_encoded = []
    
    for val in df[column_name]:
        if val not in coded_dict:
            coded_dict[val] = cter
            cter+=1
        
        id_encoded.append(coded_dict[val])
    if remove_col:
        del df[column_name]
    return id_encoded, coded_dict

# The portfolio standard cleaning process 

In [26]:
portfolio.head()

,reward,channels,difficulty,duration,offer_type,id
0,10,"[email, mobile, social]",10,7,bogo,ae264e3637204a6fb9bb56bc8210ddfd
1,10,"[web, email, mobile, social]",10,5,bogo,4d5c57ea9a6940dd891ad53e9dbe8da0
2,0,"[web, email, mobile]",0,4,informational,3f207df678b143eea3cee63160fa8bed
3,5,"[web, email, mobile]",5,7,bogo,9b98b8c7a33c4b65b9aebfe6a799e6d9
4,5,"[web, email]",20,10,discount,0b1e1539f2cc45b7b9fa7c272da2e1d7


In [25]:
# Find the columns that have nan value
portfolio.columns[portfolio.isna().mean() > 0]

Index([], dtype='object')

In [6]:
# Map hash_id to readable id
encoded_offer_ids, offer_coded_dict = id_mapper(portfolio, 'id')
portfolio['offer_id'] = encoded_offer_ids

# Create the dummies for channels column
channel_dummies = pd.get_dummies(portfolio['channels'].apply(pd.Series).stack()).sum(level=0)
cleaned_portfolio = pd.concat([portfolio, channel_dummies], axis=1)

# Rename to make columns more meaningful
cleaned_portfolio.rename({'id': 'hash_offer_id'}, axis = 1, inplace = True)

In [7]:
# Simple cleaned portfolio
cleaned_portfolio.head()

,reward,channels,difficulty,duration,offer_type,hash_offer_id,offer_id,email,mobile,social,web
0,10,"[email, mobile, social]",10,7,bogo,ae264e3637204a6fb9bb56bc8210ddfd,1,1,1,1,0
1,10,"[web, email, mobile, social]",10,5,bogo,4d5c57ea9a6940dd891ad53e9dbe8da0,2,1,1,1,1
2,0,"[web, email, mobile]",0,4,informational,3f207df678b143eea3cee63160fa8bed,3,1,1,0,1
3,5,"[web, email, mobile]",5,7,bogo,9b98b8c7a33c4b65b9aebfe6a799e6d9,4,1,1,0,1
4,5,"[web, email]",20,10,discount,0b1e1539f2cc45b7b9fa7c272da2e1d7,5,1,0,0,1
5,3,"[web, email, mobile, social]",7,7,discount,2298d6c36e964ae4a3e7e9706d1fb8c2,6,1,1,1,1
6,2,"[web, email, mobile, social]",10,10,discount,fafdcd668e3743c1bb461111dcafc2a4,7,1,1,1,1
7,0,"[email, mobile, social]",0,3,informational,5a8bc65990b245e5a138643cd4eb9837,8,1,1,1,0
8,5,"[web, email, mobile, social]",5,5,bogo,f19421c1d4aa40978ebb69ca19b0e20d,9,1,1,1,1
9,2,"[web, email, mobile]",10,7,discount,2906b810c7d4411798c6938adc9daaa5,10,1,1,0,1


# The profile standard cleaning process ###

In [8]:
profile.head()

,gender,age,id,became_member_on,income
0,None,118,68be06ca386d4c31939f3a4f0e3dd783,20170212,NaN
1,F,55,0610b486422d4921ae7d2bf64640c50b,20170715,112000.0
2,None,118,38fe809add3b4fcf9315a9694bb96ff5,20180712,NaN
3,F,75,78afa995795e4d85b5d9ceeca43f5fef,20170509,100000.0
4,None,118,a03223e636434f42ac4c3df47e8bac43,20170804,NaN


In [9]:
# Find the columns that have NaN value
profile.columns[profile.isna().mean() > 0]

Index(['gender', 'income'], dtype='object')

In [10]:
# Detected the abnormal relationship between income (NaN) and age value (118)
profile[(profile['age'] == 118) & (profile['income'].isna())].head()

,gender,age,id,became_member_on,income
0,None,118,68be06ca386d4c31939f3a4f0e3dd783,20170212,NaN
2,None,118,38fe809add3b4fcf9315a9694bb96ff5,20180712,NaN
4,None,118,a03223e636434f42ac4c3df47e8bac43,20170804,NaN
6,None,118,8ec6ce2a7e7949b1bf142def7d0e0586,20170925,NaN
7,None,118,68617ca6246f4fbc85e91a2a49552598,20171002,NaN


In [11]:
# Remove the NaN that come from the abnormal relationship
profile.dropna(subset=['income'], inplace = True)
# Reset index after drop NaN
profile.reset_index(drop=True, inplace=True)

# Convert date string to date type
profile.loc[:,'became_member_on'] = pd.to_datetime(profile.loc[:,'became_member_on'], format='%Y%m%d', errors='coerce')

# Map hash_id to readable id
encoded_customer_ids, customer_coded_dict = id_mapper(profile, 'id')
profile['customer_id'] = encoded_customer_ids

# Rename to make columns more meaningful
cleaned_profile = profile.rename({'id': 'hash_customer_id'}, axis = 1)

In [15]:
# Simple cleaned profile
cleaned_profile.head()

,gender,age,hash_customer_id,became_member_on,income,customer_id
0,F,55,0610b486422d4921ae7d2bf64640c50b,2017-07-15,112000.0,1
1,F,75,78afa995795e4d85b5d9ceeca43f5fef,2017-05-09,100000.0,2
2,M,68,e2127556f4f64592b11af22de27a7932,2018-04-26,70000.0,3
3,M,65,389bc3fa690240e798340f5a15918d5c,2018-02-09,53000.0,4
4,M,58,2eeac8d8feae4a8cad5a6af0499a211d,2017-11-11,51000.0,5


# The transcript standard cleaning process ###

In [18]:
transcript.head()

,person,event,value,time
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},0
1,a03223e636434f42ac4c3df47e8bac43,offer received,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},0
2,e2127556f4f64592b11af22de27a7932,offer received,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},0
3,8ec6ce2a7e7949b1bf142def7d0e0586,offer received,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'},0
4,68617ca6246f4fbc85e91a2a49552598,offer received,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'},0


In [27]:
# Explore value in transcript dataframe
transcript['value']

0         {'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'}
1         {'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'}
2         {'offer id': '2906b810c7d4411798c6938adc9daaa5'}
3         {'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'}
4         {'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'}
                                ...                       
306529                      {'amount': 1.5899999999999999}
306530                                    {'amount': 9.53}
306531                                    {'amount': 3.61}
306532                      {'amount': 3.5300000000000002}
306533                                    {'amount': 4.05}
Name: value, Length: 306534, dtype: object

In [21]:
#Remove abnormal customer id first to reduce cleaning data time
transcript['customer_id'] = np.nan

# Dehash the person column
# Change hash id to readable id
for idx, value in enumerate(transcript['person']):
    if value in customer_coded_dict:
        transcript.loc[idx,'customer_id'] = customer_coded_dict[value]

In [ ]:
transcript['customer_id'].head()

In [22]:
# Remove unmeaningful customer id (NaN) to reduce cleaning data time
# Drop non-exist(abnormal) user
transcript.dropna(subset=['customer_id'], inplace=True)

# Reset index after drop NA
transcript.reset_index(drop=True, inplace=True)

In [25]:
# Extract the offer id, reward, and amount value to new columns below
# Declare the desired columns then fill with nan values
transcript['hash_offer_id'] = np.nan
transcript['reward'] = np.nan
transcript['amount'] = np.nan

for i, value in enumerate(transcript['value']):
    if 'reward' in value:
        transcript.loc[i,'reward']  = value['reward']
    if 'offer id' in value:
        transcript.loc[i,'hash_offer_id']  = value['offer id']    
    if 'offer_id' in value:
        transcript.loc[i,'hash_offer_id']  = value['offer_id']
    elif 'amount' in value:
        transcript.loc[i,'amount']  = value['amount']

In [27]:
transcript.head()

,person,event,value,time,customer_id,hash_offer_id,reward,amount
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},0,2.0,9b98b8c7a33c4b65b9aebfe6a799e6d9,NaN,NaN
1,e2127556f4f64592b11af22de27a7932,offer received,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},0,3.0,2906b810c7d4411798c6938adc9daaa5,NaN,NaN
2,389bc3fa690240e798340f5a15918d5c,offer received,{'offer id': 'f19421c1d4aa40978ebb69ca19b0e20d'},0,4.0,f19421c1d4aa40978ebb69ca19b0e20d,NaN,NaN
3,2eeac8d8feae4a8cad5a6af0499a211d,offer received,{'offer id': '3f207df678b143eea3cee63160fa8bed'},0,5.0,3f207df678b143eea3cee63160fa8bed,NaN,NaN
4,aa4862eba776480b8bb9c68455b8c2e1,offer received,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},0,6.0,0b1e1539f2cc45b7b9fa7c272da2e1d7,NaN,NaN
...,...,...,...,...,...,...,...,...
272757,24f56b5e1849462093931b164eb803b5,offer completed,{'offer_id': 'fafdcd668e3743c1bb461111dcafc2a4...,714,14784.0,fafdcd668e3743c1bb461111dcafc2a4,2.0,NaN
272758,b3a1272bc9904337b331bf348c3e8c17,transaction,{'amount': 1.5899999999999999},714,14792.0,NaN,NaN,1.59
272759,68213b08d99a4ae1b0dcb72aebd9aa35,transaction,{'amount': 9.53},714,14797.0,NaN,NaN,9.53
272760,a00058cf10334a308c68e7631c529907,transaction,{'amount': 3.61},714,14810.0,NaN,NaN,3.61


In [28]:
# Map extracted hashed offer_id to readable offer_id
transcript['offer_id'] = np.nan

# Customer_id
for idx, value in enumerate(transcript['hash_offer_id']):
    if value in offer_coded_dict:
        transcript.loc[idx,'offer_id'] = offer_coded_dict[value]

# Rename
new_transcript = transcript.rename({'person': 'hash_customer_id', 'value': 'dict_value'}, axis = 1)

In [38]:
# Save dataset for further use to reduce debug time
new_transcript.to_csv('modified_transcript.csv', encoding='utf-8', index=False)

In [41]:
# Load saved dataset
df_transcript = pd.read_csv('modified_transcript.csv')
df_transcript.head()

,hash_customer_id,event,dict_value,time,customer_id,hash_offer_id,reward,amount,offer_id
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},0,2.0,9b98b8c7a33c4b65b9aebfe6a799e6d9,NaN,NaN,4.0
1,e2127556f4f64592b11af22de27a7932,offer received,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},0,3.0,2906b810c7d4411798c6938adc9daaa5,NaN,NaN,10.0
2,389bc3fa690240e798340f5a15918d5c,offer received,{'offer id': 'f19421c1d4aa40978ebb69ca19b0e20d'},0,4.0,f19421c1d4aa40978ebb69ca19b0e20d,NaN,NaN,9.0
3,2eeac8d8feae4a8cad5a6af0499a211d,offer received,{'offer id': '3f207df678b143eea3cee63160fa8bed'},0,5.0,3f207df678b143eea3cee63160fa8bed,NaN,NaN,3.0
4,aa4862eba776480b8bb9c68455b8c2e1,offer received,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},0,6.0,0b1e1539f2cc45b7b9fa7c272da2e1d7,NaN,NaN,5.0


In [94]:
# Fetch the non-transaction events
offer_transcript = new_transcript[new_transcript['event'] != 'transaction'].drop('amount', axis=1)
# Reset index after drop the transaction event
offer_transcript.reset_index(drop=True, inplace=True)
offer_transcript.head()

,event,time,customer_id,reward,offer_id
0,offer received,0,2.0,NaN,4.0
1,offer received,0,3.0,NaN,10.0
2,offer received,0,4.0,NaN,9.0
3,offer received,0,5.0,NaN,3.0
4,offer received,0,6.0,NaN,5.0


In [85]:
# Create the dictionaries that match between offer_id and offer duration
dict_portfolio = dict()
for idx, val in enumerate(portfolio['offer_id']):
    dict_portfolio[val] = portfolio.loc[idx,'duration']

# Create the offer duration for each offer id
offer_transcript['offer_duration'] = np.nan
for idx, val in enumerate(offer_transcript['offer_id']):
    if val in dict_portfolio:
        offer_transcript.loc[idx,'offer_duration'] = dict_portfolio[val]*24
offer_transcript.head()

{1: 7, 2: 5, 3: 4, 4: 7, 5: 10, 6: 7, 7: 10, 8: 3, 9: 5, 10: 7}

In [152]:
### Unfinished clear target condition
# Find the offer that each customer responds.

# Sort the offers by customer_id and time sequentially
sorted_offers = offer_transcript.sort_values(by=['customer_id','time'])

# Create unique customer id to iterate the for loop
unique_custid = sorted_offers['customer_id'].astype(int).unique()

# The dictionaries that match between the customer id and the list of offer ids
response_dict = dict() #the list of offer ids that responsed to each the customer id
offer_dict = dict() #the list of offer ids that recived the list of offer ids 

for custid in unique_custid:
    viewed_offer = []
    track_viewed_offer = []
    response_offer = []
    
    # Select dataframe of specific customer id
    df_offers = offer_transcript[offer_transcript['customer_id'] == custid]
    df_offers.reset_index(drop=True,inplace=True)
    
    for idx, event in enumerate(df_offers['event']):
        if event == 'offer received':
            # Store the recieved offer
            viewed_offer.append(df_offers.loc[idx, 'offer_id'])
        elif event == 'offer viewed':
            # if offer not in track_viewed_offer add it
            if not df_offers.loc[idx, 'offer_id'] in track_viewed_offer:
                track_viewed_offer.append(df_offers.loc[idx, 'offer_id'])
        elif event == 'offer completed':
            # Offer respond when customer both view the offer and then completed offer
            if df_offers.loc[idx, 'offer_id'] in track_viewed_offer:
                response_offer.append(df_offers.loc[idx, 'offer_id'])
                track_viewed_offer.remove(df_offers.loc[idx, 'offer_id'])
                
    # Create Dictionaries for current customer_id
    response_dict[custid] = response_offer
    offer_dict[custid] = viewed_offer

In [213]:
# Create response (target) columns by conditons 
new_profile['response'] = np.nan

#The number of offer
num=0

for custid in offer_dict:
    for offer_id in offer_dict[custid]:
        df_profile = new_profile[new_profile['customer_id'] == custid].reset_index(drop=True)
        df_portfolio = cleaned_portfolio[cleaned_portfolio['offer_id'] == offer_id].reset_index(drop=True)
        if num==0:
            features = pd.concat([df_profile,df_portfolio], axis=1)
        else:
            new_features = pd.concat([df_profile,df_portfolio], axis=1)
            features = pd.concat([features,new_features], axis=0).reset_index(drop=True)
        # Assign response or not response to column
        if offer_id in response_dict[custid]:
            features.loc[num, 'response'] = 1
        else:
            #print(i)
            features.loc[num, 'response'] = 0
        num+=1
features.head()

,gender,age,became_member_on,income,customer_id,response,reward,channels,difficulty,duration,offer_type,hash_offer_id,offer_id,email,mobile,social,web
0,F,55,2017-07-15,112000.0,1,0.0,5,"[web, email, mobile]",5,7,bogo,9b98b8c7a33c4b65b9aebfe6a799e6d9,4,1,1,0,1
1,F,55,2017-07-15,112000.0,1,0.0,0,"[web, email, mobile]",0,4,informational,3f207df678b143eea3cee63160fa8bed,3,1,1,0,1
2,F,75,2017-05-09,100000.0,2,1.0,5,"[web, email, mobile]",5,7,bogo,9b98b8c7a33c4b65b9aebfe6a799e6d9,4,1,1,0,1
3,F,75,2017-05-09,100000.0,2,0.0,0,"[email, mobile, social]",0,3,informational,5a8bc65990b245e5a138643cd4eb9837,8,1,1,1,0
4,F,75,2017-05-09,100000.0,2,1.0,10,"[email, mobile, social]",10,7,bogo,ae264e3637204a6fb9bb56bc8210ddfd,1,1,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66496,F,83,2016-03-07,50000.0,14824,0.0,10,"[email, mobile, social]",10,7,bogo,ae264e3637204a6fb9bb56bc8210ddfd,1,1,1,1,0
66497,F,62,2017-07-22,82000.0,14825,1.0,3,"[web, email, mobile, social]",7,7,discount,2298d6c36e964ae4a3e7e9706d1fb8c2,6,1,1,1,1
66498,F,62,2017-07-22,82000.0,14825,0.0,0,"[web, email, mobile]",0,4,informational,3f207df678b143eea3cee63160fa8bed,3,1,1,0,1
66499,F,62,2017-07-22,82000.0,14825,0.0,5,"[web, email, mobile, social]",5,5,bogo,f19421c1d4aa40978ebb69ca19b0e20d,9,1,1,1,1


In [ ]:
# Save the dataset to reduce debug time
features.to_csv('features.csv', encoding='utf-8', index=False)

In [ ]:
# Load the saved dataset
df_features = pd.read_csv('features.csv')
df_features.head()

In [10]:
df_features.head()

,gender,age,became_member_on,income,customer_id,response,reward,channels,difficulty,duration,offer_type,hash_offer_id,offer_id,email,mobile,social,web
0,F,55,2017-07-15,112000.0,1,0.0,5,"['web', 'email', 'mobile']",5,7,bogo,9b98b8c7a33c4b65b9aebfe6a799e6d9,4,1,1,0,1
1,F,55,2017-07-15,112000.0,1,0.0,0,"['web', 'email', 'mobile']",0,4,informational,3f207df678b143eea3cee63160fa8bed,3,1,1,0,1
2,F,75,2017-05-09,100000.0,2,1.0,5,"['web', 'email', 'mobile']",5,7,bogo,9b98b8c7a33c4b65b9aebfe6a799e6d9,4,1,1,0,1
3,F,75,2017-05-09,100000.0,2,0.0,0,"['email', 'mobile', 'social']",0,3,informational,5a8bc65990b245e5a138643cd4eb9837,8,1,1,1,0
4,F,75,2017-05-09,100000.0,2,1.0,10,"['email', 'mobile', 'social']",10,7,bogo,ae264e3637204a6fb9bb56bc8210ddfd,1,1,1,1,0


## Save merged dataset to CSV file

In [11]:
# drop unneeded columns
df_features.drop(['channels','hash_offer_id'], axis=1, inplace=True)
df_features.head()

,gender,age,became_member_on,income,customer_id,response,reward,difficulty,duration,offer_type,offer_id,email,mobile,social,web
0,F,55,2017-07-15,112000.0,1,0.0,5,5,7,bogo,4,1,1,0,1
1,F,55,2017-07-15,112000.0,1,0.0,0,0,4,informational,3,1,1,0,1
2,F,75,2017-05-09,100000.0,2,1.0,5,5,7,bogo,4,1,1,0,1
3,F,75,2017-05-09,100000.0,2,0.0,0,0,3,informational,8,1,1,1,0
4,F,75,2017-05-09,100000.0,2,1.0,10,10,7,bogo,1,1,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66496,F,83,2016-03-07,50000.0,14824,0.0,10,10,7,bogo,1,1,1,1,0
66497,F,62,2017-07-22,82000.0,14825,1.0,3,7,7,discount,6,1,1,1,1
66498,F,62,2017-07-22,82000.0,14825,0.0,0,0,4,informational,3,1,1,0,1
66499,F,62,2017-07-22,82000.0,14825,0.0,5,5,5,bogo,9,1,1,1,1


In [17]:
# Rearrange columns according to the list below
cols = ['customer_id','gender','age','became_member_on','income','offer_id','reward',
        'difficulty','duration','offer_type','email','mobile','social','web','response']
df = df_features.loc[:, cols]
df

,customer_id,gender,age,became_member_on,income,offer_id,reward,difficulty,duration,offer_type,email,mobile,social,web,response
0,1,F,55,2017-07-15,112000.0,4,5,5,7,bogo,1,1,0,1,0.0
1,1,F,55,2017-07-15,112000.0,3,0,0,4,informational,1,1,0,1,0.0
2,2,F,75,2017-05-09,100000.0,4,5,5,7,bogo,1,1,0,1,1.0
3,2,F,75,2017-05-09,100000.0,8,0,0,3,informational,1,1,1,0,0.0
4,2,F,75,2017-05-09,100000.0,1,10,10,7,bogo,1,1,1,0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66496,14824,F,83,2016-03-07,50000.0,1,10,10,7,bogo,1,1,1,0,0.0
66497,14825,F,62,2017-07-22,82000.0,6,3,7,7,discount,1,1,1,1,1.0
66498,14825,F,62,2017-07-22,82000.0,3,0,0,4,informational,1,1,0,1,0.0
66499,14825,F,62,2017-07-22,82000.0,9,5,5,5,bogo,1,1,1,1,0.0
